In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain. prompts import PromptTemplate
from langchain. chains import LLMChain
from langchain_community.document_loaders import CSVLoader
import PyPDF2
#from transformers import GPT2TokenizerFast
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken


In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv


C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv(dotenv_path='.env')

# https://platform.openai.com/api-keys
openai_api_key = os.getenv("OPENAI_API_KEY")

# https://console.groq.com/keys
groq_api_key = os.getenv("GROQ_API_KEY")

# https://console.anthropic.com/dashboard
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# https://aistudio.google.com/app/apikey
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [6]:
# Abrir o arquivo PDF
def ler_pdf(pdf_path):
# Verificar se o arquivo existe no caminho especificado
    if os.path.exists(pdf_path):
        print(f"Arquivo {pdf_path} carregado com sucesso!")
    else:
        print(f"O arquivo {pdf_path} não foi encontrado. Verifique o caminho e tente novamente.")
        return None

    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
    
        all_text = ""
    
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text = page.extract_text()
            if text:
                all_text += text
            
    return all_text

# Abrir o arquivo TXT
def ler_txt(txt_path):
# Verificar se o arquivo existe no caminho especificado
    if os.path.exists(txt_path):
        print(f"Arquivo {txt_path} carregado com sucesso!")
    else:
        print(f"O arquivo {txt_path} não foi encontrado. Verifique o caminho e tente novamente.")
        return None
    
    with open(txt_path, 'r', encoding='utf-8') as file:
        all_text = file.read()

    return all_text

name1 = "ABC_Evolucao_redux.pdf"
arquivo = f"dados/{name1}"  # Especifique o caminho do PDF
text1 = ler_pdf(arquivo)

name2 = "selecao#.txt"
arquivo = f"dados/{name2}"  # Especifique o caminho do PDF
text2 = ler_txt(arquivo)

text = text1
print(text)

Arquivo dados/ABC_Evolucao_redux.pdf carregado com sucesso!
Arquivo dados/selecao#.txt carregado com sucesso!
A E volução 
é Fato 
A E volução 
é Fato Diretoria da Academia Brasileira de Ciências 
Triênio 2022-2025 
Presidente
Helena Bonciani Nader
Vice-Presidente
Jailson Bittencourt de Andrade
Vice-Presidentes Regionais
Norte: Adalberto Luis Val 
Nordeste e Espírito Santo: Jailson Bittencourt de Andrade
Minas Gerais e Centro-Oeste: Virgílio Augusto Fernandes Almeida 
Rio de Janeiro: Maria Domingues Vargas 
São Paulo: Glaucius Oliva
Sul: Ruben George Oliven
Diretores
Alvaro Toubes Prata 
Maria Domingues Vargas 
Mariangela Hungria
Roberto Lent
Virgílio Augusto Fernandes Almeida
MEMBROS INSTITUCIONAIS 
APOIOS INSTITUCIONAIS DIAMANTE OURO 
 PRATA 
BRONZE 
A E volução 
é Fato 
Coordenador 
Carlos Frederico Martins Menck 
Rio de Janeiro, 2024 © Direitos autorais, 2024, de publicação, de organização, da 
Academia Brasileira de Ciências
Rua Anﬁlóﬁo de Carvalho, 29 - 3º Andar
20030-060 - Rio d

In [7]:
tokenizer = tiktoken.get_encoding("cl100k_base")
#tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", use_ssl=False)  # este gpt2 é gratuito para calcular número de tokens, não precisa de api_key

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter( # divide o PDF em blocos/chunks de 512 tokens
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
    separators=["#","\n"] #  ,
)

chunks1 = []
chunks2 = []

# chunks = text_splitter.create_documents([text])
metadata = {"source": name1,"row": 0}
chunks1 = text_splitter.create_documents([text1], metadatas=[metadata])

metadata = {"source": name2,"row": 0}
chunks2 = text_splitter.create_documents([text2], metadatas=[metadata])

combined_chunks = chunks1 + chunks2

i = 0
for chunk in combined_chunks:
    print(f"Texto [{i}]: {chunk.page_content}\nMetadados: {chunk.metadata}\n")
    i = i + 1

Texto [0]: A E volução 
é Fato 
A E volução 
é Fato Diretoria da Academia Brasileira de Ciências 
Triênio 2022-2025 
Presidente
Helena Bonciani Nader
Vice-Presidente
Jailson Bittencourt de Andrade
Vice-Presidentes Regionais
Norte: Adalberto Luis Val 
Nordeste e Espírito Santo: Jailson Bittencourt de Andrade
Minas Gerais e Centro-Oeste: Virgílio Augusto Fernandes Almeida 
Rio de Janeiro: Maria Domingues Vargas 
São Paulo: Glaucius Oliva
Sul: Ruben George Oliven
Diretores
Alvaro Toubes Prata 
Maria Domingues Vargas 
Mariangela Hungria
Roberto Lent
Virgílio Augusto Fernandes Almeida
MEMBROS INSTITUCIONAIS 
APOIOS INSTITUCIONAIS DIAMANTE OURO 
 PRATA 
BRONZE 
A E volução 
é Fato 
Coordenador 
Carlos Frederico Martins Menck 
Rio de Janeiro, 2024 © Direitos autorais, 2024, de publicação, de organização, da 
Academia Brasileira de Ciências
Rua Anﬁlóﬁo de Carvalho, 29 - 3º Andar
20030-060 - Rio de Janeiro, RJ - Brasil
Tel: +55 21 3907-8100 
www.abc.org.br
Assessoria
Marcos Cortesão Barnsley Sc

In [8]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
# pip install faiss-cpu

#loader = CSVLoader(file_path="output.csv", encoding='utf-8')
#documents = loader.load()
#documents

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
db = FAISS.from_documents(combined_chunks, embeddings)

In [ ]:
chunks1

In [11]:
question = "Quando ocorreu a “explosão do Cambriano” ?"
similar_response = db.similarity_search(question,k=3)
best_practice = [doc.page_content + " # " + doc.metadata['source'] for doc in similar_response]
best_practice

['a Terra e, posteriormente, decantou. Além do irídio, estruturas como \nesférulas de basalto e grãos alterados de quartzo, que são diagnósti-\ncas de impactos, reforçam o cenário. Alguns autores defendem que A Evolução é Fato \n179 Figura 7.  Cratera de Chickxulub, no Golfo do \nMéxico, local do impacto do corpo celeste \nhá 66 milhões de anos. (Fonte: ilustração au-\ntoral elaborada para a ABC/Fábio Andriolo)\no ﬁnal do Cretáceo vinha passando por uma crise climática global, \nenvolvendo chuvas ácidas, privação de oxigênio e alterações climá-\nticas, desencadeada por um grande evento de vulcanismo na Índia, \ndenominado Deccan Traps, relacionado à fragmentação da Pangeia. \nEntretanto, o meteorito de Chicxulub segue como a causa principal \ndessa grande extinção.\nAssim como em todo evento de extinção em massa, não se sabe \nbem o porquê de todos os dinossauros não-avianos terem sido  \nextintos e tampouco o porquê de mamíferos e aves (dinossauros avia-\nnos) estarem até hoje no plan

In [12]:
similar_response

[Document(metadata={'source': 'ABC_Evolucao_redux.pdf', 'row': 0}, page_content='a Terra e, posteriormente, decantou. Além do irídio, estruturas como \nesférulas de basalto e grãos alterados de quartzo, que são diagnósti-\ncas de impactos, reforçam o cenário. Alguns autores defendem que A Evolução é Fato \n179 Figura 7.  Cratera de Chickxulub, no Golfo do \nMéxico, local do impacto do corpo celeste \nhá 66 milhões de anos. (Fonte: ilustração au-\ntoral elaborada para a ABC/Fábio Andriolo)\no ﬁnal do Cretáceo vinha passando por uma crise climática global, \nenvolvendo chuvas ácidas, privação de oxigênio e alterações climá-\nticas, desencadeada por um grande evento de vulcanismo na Índia, \ndenominado Deccan Traps, relacionado à fragmentação da Pangeia. \nEntretanto, o meteorito de Chicxulub segue como a causa principal \ndessa grande extinção.\nAssim como em todo evento de extinção em massa, não se sabe \nbem o porquê de todos os dinossauros não-avianos terem sido  \nextintos e tampouco

In [13]:
template = """
Você é um assistente virtual responde perguntas sobre a evolução humana.
Sua função será responder e-mails que recebemos de requerentes.
Siga todas as regras abaixo:
1. Você deve buscar se comportar de maneira cordial e solícita.
2. Suas respostas devem ser bem didáticas.
3. Evite informações irrelevantes. Preste atenção apenas no conteúdo útil da mensagem.
Aqui está uma mensagem de uma dúvida recebida de um novo requerente.
{message}
Aqui está uma lista de documentos relevantes para responder esta dúvida
{best_practice}
Escreva a melhor resposta que eu deveria enviar para solucionar a dúvida apresentada pela pergunta deste requerente.
Encerre com a seguinte assinatura:

Equipe Fale Conosco 
email: faleconosco@teste.com
"""

In [14]:
from langchain.prompts.prompt import PromptTemplate
prompt = PromptTemplate(input_variables=['message','best_practice'],template=template)

In [15]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
chain = prompt | llm
response = chain.invoke({"message":question,"best_practice":best_practice})
print(response.content)


Prezado(a) requerente,

A "explosão Cambriana" refere-se a um período significativo na história da vida na Terra, que ocorreu entre aproximadamente 538 e 485 milhões de anos atrás. Durante esse intervalo de tempo, houve um aumento extraordinário na diversidade de organismos multicelulares, incluindo o surgimento de muitos dos filos modernos de animais.

Este evento é notável por ter introduzido uma variedade de formas de vida complexas, que incluíam organismos com carapaças e esqueletos mineralizados, oferecendo estratégias de defesa contra predadores recém-aparecidos. Essa rápida diversificação é frequentemente atribuída a mudanças ambientais que permitiram novas interações ecológicas, como a predação e o pastoreio, que tiveram um impacto profundo na evolução dos organismos.

Se você tiver mais perguntas ou precisar de esclarecimentos adicionais, fique à vontade para entrar em contato.

Atenciosamente,

Equipe Fale Conosco  
email: faleconosco@teste.com


In [16]:
response

AIMessage(content='Prezado(a) requerente,\n\nA "explosão Cambriana" refere-se a um período significativo na história da vida na Terra, que ocorreu entre aproximadamente 538 e 485 milhões de anos atrás. Durante esse intervalo de tempo, houve um aumento extraordinário na diversidade de organismos multicelulares, incluindo o surgimento de muitos dos filos modernos de animais.\n\nEste evento é notável por ter introduzido uma variedade de formas de vida complexas, que incluíam organismos com carapaças e esqueletos mineralizados, oferecendo estratégias de defesa contra predadores recém-aparecidos. Essa rápida diversificação é frequentemente atribuída a mudanças ambientais que permitiram novas interações ecológicas, como a predação e o pastoreio, que tiveram um impacto profundo na evolução dos organismos.\n\nSe você tiver mais perguntas ou precisar de esclarecimentos adicionais, fique à vontade para entrar em contato.\n\nAtenciosamente,\n\nEquipe Fale Conosco  \nemail: faleconosco@teste.com',